# Background subspace inhibition with IBCM neurons
A layer of IBCM neurons is used to inhibit the activity of ORNs in response to a fluctuating olfactory background. Synaptic weights from the input layer to the inhibition layer, $M = (\vec{m}_1, \vec{m}_2, \ldots)$, are learnt according to the IBCM rule. Synaptic weights for inhbition, from the inhibitory neurons to the projection layer, are learnt to minimize the squared norm of the projection neuron (PN) layer. In this way, the network of IBCM neurons is like an autoencoder applying feedforward inhibition. 

![test](figures/feedforward_inhibitory_network.png)

## Network of IBCM neurons

Consider a feedforward network of IBCM  neurons. Each will be paired with an inhibitory neuron, but those inhibitory neurons will not interact with other neurons except their own IBCM neuron, so we can discuss them later. Each IBCM neuron has $N_D \geq N_I$ input synapses, represented by the connectivity vector $\vec{m}_i = (m^1_i, \ldots, m^{N_D}_i)$. Its activation upon stimulation, before coupling to other IBCM neurons is considered, is given by $c = \vec{m}_i \cdot \vec{x}$, where $\vec{x}$ is a two-dimensional input vector. Its activity inhibited by other IBCM neurons in the network is 

$$ \bar{c}_i = c_i - \eta \sum_{j \neq i} c_j \quad \mathrm{where} \, \, c_i(t) = \vec{m}_i(t) \cdot \vec{x}(t)  \, \, .$$

The update equation of each IBCM neuron's weights uses this inhibited activity:

$$ \dot{m}_i = \mu \left[ \bar{c}_i(\bar{c}_i - \bar{\Theta}_i) \vec{x} - \eta \sum_{j \neq i} \bar{c}_j(\bar{c}_j - \bar{\Theta}_j) \vec{x} \right]  \quad \mathrm{where} \, \, \bar{\Theta}_i = \mathbb{E}[\bar{c}_i^2] $$

The parameter $\eta$ is the coupling strength. The thresholds $\bar{\Theta}_i$ are time-dependent averages of the IBCM neuron's inhibited activity squared, $\bar{c}_i^2$; they evolve with $\vec{m}_i$ according to the differential equation

$$ \dot{\bar{\Theta}}_i = \frac{1}{\tau_\Theta} (\bar{c}_i^2 - \bar{\Theta}_i)  \,\, .$$

Hence, they effectively take the average of $\bar{c}_i^2$ over a sliding exponential window with a time scale $\tau_\Theta$. This time scale should be a lot longer than the fluctuation time scale of the input, $\tau_c$, but also a lot faster than the slow time scale of $\vec{m}_i$ learning itself. In short, we should have $\tau_c \ll \tau_{\Theta} \ll \frac{1}{\mu}$. 

## Lateral inhibition with the IBCM neurons
Use the IBCM neurons as lateral inhibitory neurons. There are weights $\vec{w}_i$ going from IBCM neuron $i$ to projection neurons. We can store them in the matrix $W$, where each column is a vector $\vec{w}_i$. The total inhibition received by projection neurons is therefore $\sum_{j} \bar{c}_j \vec{w}_j = W \vec{\bar{c}}$; if these neurons have an element-wise activation function $R$, typically a ReLU function, they take the value

$$ \vec{s} = R\left(\vec{x}_{in}(t) - W \vec{\bar{c}}  \right) $$

By calculating the gradient of the cost function

$$ C(\vec{w}_j) = \frac12 \mathbb{E}\left[\vec{s}^T \vec{s} \right] + \frac{\beta}{2\alpha} \mathbb{E}\left[ \vec{w}_j^T \vec{w}_j \right] $$

we find that the inhibition weights leaving neuron $j$ should evolve according to

$$ \frac{d \vec{w}_j}{dt} = -\alpha \nabla_{\vec{w}_j} C(\vec{w}_j) = \alpha \bar{c}_j \vec{s} R'\left(\vec{s}\right) -  \beta \vec{w}_j $$

after assuming that $\alpha$ is small enough to take instantaneous values but still see $\vec{w}$ converge on average to the optimum of the cost function. $R'\left(\vec{s}\right)$ is the element-wise derivative of the activation function; for instance, it is $1$ if $R$ is the identity function, or a Heaviside function if $R$ is a ReLU. In those two cases, it can simply be omitted -- in the latter case, the ReLU applied on $\vec{s}$ itself ensures the term is zero if the difference $\vec{x} - W\vec{\bar{c}}$ is negative. 
Here, I will use a ReLU, so

$$ \frac{d\vec{w}_j}{dt} = \alpha \bar{c}_j \vec{s} -  \beta \vec{w}_j$$

In [ ]:
import numpy as np
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from time import perf_counter

from utils.statistics import seed_from_gen

In [ ]:
from modelfcts.ibcm import (
    integrate_inhib_ibcm_network, 
    relu_inplace, 
    compute_mbars_cgammas_cbargammas
)

## Input process
To have multiple input components fluctuating realistically, we write the input as
$$ \vec{x}(t) = \sum_{\alpha=1}^K \nu_\alpha \vec{x}_\alpha $$
where the $\nu_\alpha$ are random variables with some correlation, to mimick the turbulent flow that carries all odorants together. Ideally, they would follow the distributions derived in (Celani, Villermaux and Vergassola 2014), but those are a bit tricky to simulate.

### General case of the Ornstein-Uhlenbeck process
The multivariate Langevin equation for the Ornstein-Uhlenbeck process is:

$$ d\vec{x} = -A \vec{x}(t) dt + B dW(t) $$

where $\frac{dW}{dt} = \vec{\eta}(t)$, a vector of gaussian white noise (independent components), $A$ and $B$ are matrices. Assume the matrix $A$ is normal and can be diagonalized as $A = U D U^\dagger$, where $D = \mathrm{diag}(\lambda^1, ..., \lambda^n)$. For a deterministic initial condition $\vec{x}(t_0) = \vec{x}_0$, the general solution is that $\vec{x}(t)$ follows a multivariate normal distribution, with mean and variance given by

$$ \langle \vec{x}(t) \rangle = U\mathrm{e}^{-D(t-t_0)}U^{\dagger} \vec{x}_0 $$
$$ \langle \vec{x}(t) \vec{x}(t)^T \rangle = U J(t, t_0) U^\dagger $$

where the components of $J$ are 

$$ J^{ij}(t, t_0) = \left(\frac{U^\dagger B B^T U}{\lambda^i + \lambda^j} \right)^{ij} \left(1 - e^{-(\lambda^i + \lambda^j)(t - t_0)}  \right) $$


The stationary distribution of $\vec{x}$ is thus

$$ \vec{x}^* \sim \mathcal{N} \left(\vec{0}, \left(\frac{B B^T}{\lambda^k + \lambda^l} \right)^{kl} \right) \,\, .$$

For a non-zero mean, simulate the zero-mean process and add the average afterwards, it's simpler. 

For details, see Gardiner, chapter 4.2.6. 

### Exact numerical simulation, general case
To simulate a realization of this process exactly, we use a trick suggested by Gillespie in the univariate case (which only works for the Ornstein-Uhlenbeck process because it's linear and gaussian). We iteratively take $\vec{x}(t)$ as the initial condition of the evolution up to $\vec{x}(t + \Delta t)$, the distribution of which is

$$ \vec{x}(t + \Delta t) \sim \mathcal{N}\left( U e^{-D \Delta t}U^\dagger \vec{x}(t) , U J(t + \Delta t, t) U^\dagger \right) $$

which can be rewritten using the following property of multivariate normal distributions: if $\vec{n} \sim \mathcal{N}(\vec{0}, \mathbb{1})$, then $\vec{x} = \vec{\mu} + \Psi \vec{n} \sim \mathcal{N}(\vec{\mu}, \Psi \Psi^T)$ ($\Psi$ is the Cholesky decomposition of the desired covariance matrix). This property is easily demonstrated by computing $\langle \vec{x} \rangle$ and $\langle \vec{x} \vec{x}^T \rangle$ and using the linearity of multivariate normal distributions. For our update rule, this gives

$$ \vec{x}(t + \Delta t) = U e^{-D \Delta t}U^\dagger \vec{x}(t) + \mathrm{Chol}\left[U J(t + \Delta t, t) U^\dagger \right] \cdot \vec{n} $$

where $\vec{n}$ is a vector of standard normal(0, 1) samples. The matrices $U e^{-D \Delta t}U^\dagger$ and $\mathrm{Chol}\left[U J(t + \Delta t, t) U^\dagger \right]$ can be computed only once and applied repeatedly to the $\vec{x}(t)$ obtained in sequence and the $\vec{n}$ drawn at each iteration. The Cholesky decomposition of $UJU^\dagger$ is not obviously expressed in terms of $B$, because the possibly different $\lambda^i$ values mix up components. 


### Simple case and exact simulation of it
If $A$ is diagonal, the $U$ matrices are just identity matrices and disappear, but the Cholesky decomposition of $J(t + \Delta t, t)$ is still not obvious. More explicit expressions can be obtained in the simplifying case where $A$ is proportional to the identity matrix, i.e., all components of $\vec{x}$ have the same fluctuation time scale. 

Let's say that $A =  \frac{1}{\tau} \mathbb{1}$, where $\tau$ is the fluctuation time scale ($\lambda^i = \tau \,\, \forall i$). Then, the matrix $J$ simplifies to 

$$J(t, t_0) = \frac{\tau}{2}\left(1 - e^{-2(t - t_0)/\tau} \right)  BB^T  $$

and its Cholesky decomposition is simply $\sqrt{\frac{\tau}{2}\left(1 - e^{-2(t - t_0)/\tau} \right) } B$. Hence, the distribution of $\vec{x}(t)$ at any time since $t_0$ (deterministic initial condition $\vec{x}_0$) is

$$ \vec{x}(t) \sim \mathcal{N} \left(e^{-(t-t_0)/\tau} \vec{x}_0, \frac{\tau}{2}\left(1 - e^{-2(t - t_0)/\tau} \right)  BB^T  \right) $$

The stationary distribution is simply the above with the exponential factors set to 0. The update rule from $\vec{x}(t)$ to $\vec{x}(t + \Delta t)$ to simulate a realization of the process is nicer as well:

$$ \vec{x}(t + \Delta t) = e^{-\Delta t / \tau} \vec{x}(t) + \sqrt{\frac{\tau}{2} \left(1 - e^{-2\Delta t/\tau}  \right)} B \cdot \vec{n} $$

where $\vec{n} \sim \mathcal{N}(\vec{0}, \mathbb{1})$ is a vector of independent standard normal samples.

As before, we can compute once the (scalar) factor $e^{-\Delta t / \tau}$ and the . This is exact for any $\Delta t$, there is no increase in accuracy by decreasing $\Delta t$. You just choose the $\Delta t$ resolution at which you want to sample the realization of the process. 

### Symmetric choices for correlations
We want all pairs of $\nu_\gamma$ to have the same correlation. More specifically, we want to force a Pearson correlation coefficient of $0 < \rho < 1$ between any pair of $\nu$s. We suppose all background components have the same individual variance $\sigma^2$. The corresponding covariance matrix we want for the steady-state distribution is

$$ \Sigma = \sigma^2 \begin{pmatrix}
    1 & \rho & \ldots & \rho \\
    \rho & 1 & \ldots & \rho \\
    \ldots & \ldots & \ldots & \ldots \\
    \rho & \rho & \ldots & 1
\end{pmatrix} $$

If we apply Cholesky decomposition to get $\Sigma = \Psi \Psi^T$, then $\sqrt{\tau/2} B = \Psi$, since the steady-state covariance of the Ornstein-Uhlenbeck process is, in this simplified case, $\frac{\tau}{2} BB^T$. The $M_B$ coefficient in the update rule is then

$$ M_B = \sqrt{\tau/2(1 - e^{-2 \Delta t/\tau})}B = \sqrt{(1 - e^{-2 \Delta t/\tau})} \Psi $$

The other coefficient is just

$$ M_A = e^{-\Delta t / \tau} \mathbb{1} $$

In [ ]:
# Functions to update the fluctuating background variable
from modelfcts.backgrounds import update_ou_kinputs, update_ou_2inputs, decompose_nonorthogonal_basis

# Run a simulation with gaussian background
Select parameters, background components below to integrate the IBCM and inhibitory neuron equations while the background fluctuates. One simulation runs in ~10 s on my laptop (not very efficient Python code). 

### Analytical fixed point prediction for IBCM model 
**Consider the case of gaussian $\nu_{\alpha}$ and $\rho=0$**

Calculations predicting a $K-2$ dimensional ensemble of fixed points in the subspace spanned by the $K$ background components. If $D > K$, the $D-K$ dimensions not spanned by the odor components have no dynamics happening in them,  since $\frac{d\vec{m}}{dt} \propto \vec{x}(t)$; initial conditions remain unchanged in that space. So, in the full $D$-dimensional space, the fixed points occupy a structure of $D-2$ dimensions, but only $K-2$ of them are not trivial. 
The two constraints satisfied at the fixed points are, in the $\rho = 0$ case, which is just a linear transformation away from the more general case (by diagonalization of the covariance matrix):
$$ \sum_{\alpha} \bar{c}_{\alpha} = 1 $$
$$ \sum_{\alpha} \bar{c}_{\alpha}^2 = \frac{1}{\sigma^2} $$
where
$$ \bar{c}_{i, \alpha} = \vec{m}_{i} \cdot \vec{x}_{\alpha} - \eta \sum_{j \neq i} \vec{m}_j \cdot \vec{x}_{\alpha} $$

Note that the case $\rho \neq 0$ would be qualitatively the same as $\rho = 0$: we could always rewrite the input mixture as a linear combination in the basis of the eigenvectors of the correlation matrix, where there are no correlations between components. Hence, posing $\rho = 0$ to simplify calculations does not reduce the generality of the result. 

In the $\rho \neq 0$, one can derive the fixed point by transforming to normal coordinates, applying those two constraints on the new odor components, and transforming back. I did not take the time to do it explicity. Here, for comparison with the calculation, I set $\rho = 0$ and compare to analytical predictions. 

In [ ]:
### General simulation parameters
n_dimensions = 4
n_components = 3
n_neurons = 6

# Simulation time scales
duration = 100000.0
deltat = 1.0
learnrate = 0.003
tau_avg = 150
inhib_rates = [0.00025, 0.00005]  # alpha, beta

# coupling strength
coupling_eta = 0.2 / n_neurons
ibcm_rates = [learnrate, tau_avg, coupling_eta]

# Choose three LI vectors in (+, +, +) octant: [0.8, 0.1, 0.1], [0.1, 0.8, 0.1], etc.
back_components = 0.1*np.ones([n_components, n_dimensions])
for i in range(n_components):
    if i < n_dimensions:
        back_components[i, i] = 0.8
    else:  # If there are more components than there are dimensions (ORNs)
        back_components[i, i % n_dimensions] = 0.8 - i
    # Normalize
    back_components[i] = back_components[i] / np.sqrt(np.sum(back_components[i]**2))

# Initial synaptic weights: small positive noise near origin
rgen_meta = np.random.default_rng(seed=0x8549e8fc7718e5fa3e7516addff73b86)
init_synapses = 0.8*rgen_meta.random(size=[n_neurons, n_dimensions])

# Initial background vector and initial nu values
averages_nu = np.ones(n_components) / np.sqrt(n_components)
init_nu = np.zeros(n_components)
init_bkvec = averages_nu.dot(back_components)
# Initial background params, ordered with nu first for the update_ou_kinputs function
init_back_list = [init_nu, init_bkvec]

## Compute the matrices in the Ornstein-Uhlenbeck update equation
# Update matrix for the mean term: 
# Exponential decay with time scale tau_nu over time deltat
tau_nu = 2.0  # Correlation time scale of the background nu_gammas (same for all)
update_mat_A = np.identity(n_components)*np.exp(-deltat/tau_nu)

# Steady-state covariance matrix
sigma2 = 0.09
correl_rho = 0.0  # Set to zero for comparison with analytical prediction
steady_covmat = correl_rho * sigma2 * np.ones([n_components, n_components])  # Off-diagonals: rho
steady_covmat[np.eye(n_components, dtype=bool)] = sigma2  # diagonal: ones

# Cholesky decomposition of steady_covmat gives sqrt(tau/2) B
# Update matrix for the noise term: \sqrt(tau/2(1 - exp(-2*deltat/tau))) B
psi_mat = np.linalg.cholesky(steady_covmat)
update_mat_B = np.sqrt(1.0 - np.exp(-2.0*deltat/tau_nu)) * psi_mat

back_params = [update_mat_A, update_mat_B, back_components, averages_nu]

In [ ]:
# m_init, update_bk, bk_init, ibcm_params, inhib_params, bk_params, tmax, dt, seed=None, noisetype="normal"
sim_results = integrate_inhib_ibcm_network(init_synapses, update_ou_kinputs, init_back_list, ibcm_rates, 
                inhib_rates, back_params, duration, deltat, seed=seed_from_gen(rgen_meta), noisetype="normal")
# tseries, bk_series, bkvec_series, m_series, cbar_series, w_series, s_series
tser, nuser, bkvecser, mser, cbarser, _, wser, sser = sim_results

## Plots of the solution

For a gaussian background, there is degeneracy in the fixed points of IBCM neurons. 
We plot a few different quantities to control how the model is behaving:
- The sums $\sum_{\gamma} c_{\gamma}$ and $\sum_{\gamma} c_{\gamma}^2$, for which analytical values are $\frac{1}{\langle \nu \rangle}$ and $\frac{1}{\sigma^2}$, respectively. 
- The inhibitory weights $\vec{w}^j$ leaving each IBCM neuron. The steady-states can't be predicted, really, because of the degeneracy (TODO: check if we can predict their sums or their norms anyways, etc.). 


Because of the degeneracy, the $\bar{m}_{\gamma}$s individuallyn can take arbitrary values as the $\vec{\bar{m}}$ land anywhere on the N-2 dimensional sphere of fixed points, so plotting them separately as a function of time is not very informative. However, a 3D plot of the $\vec{\bar{m}}$ is illustrative, so we keep doing that below. 



### IBCM neurons
Dynamics, comparison to analytical fixed points, and plot of the time course of some selected components of $\vec{m}$ of a few vectors. 

In [ ]:
# Calculate cgammas_bar and mbars
transient = 50000
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser, c_gammas, cbars_gamma = compute_mbars_cgammas_cbargammas(mser, coupling_eta, back_components)

sums_cbars_gamma = np.sum(cbars_gamma, axis=2)
sums_cbars_gamma2 = np.sum(cbars_gamma*cbars_gamma, axis=2)

# Constaint 1: sum of c_gammas for each neuron
print("Comparison to analytical fixed points")
print("This should be all ones:", np.mean(sums_cbars_gamma[transient:], axis=0) * averages_nu.mean())
# Constraint 2: sum of c_gammas^2 for each neuron, copmared to 1/sigma^2
print("This should be all zeros:", np.mean(sums_cbars_gamma2[transient:], axis=0)*sigma2 - 1.0)

In [ ]:
from simulfcts.plotting import plot_cbars_gammas_sums

In [ ]:
fig, axes = plot_cbars_gammas_sums(tser, sums_cbars_gamma, sums_cbars_gamma2, skp=200, skp_lbl=1)
axes[0].axhline(1.0 / averages_nu.mean(), ls="--", color="k", label=r"$1 / \langle \nu \rangle$")
axes[1].axhline(1.0 / sigma2, ls="-.", color="k", label=r'$1 / \sigma^2$')
axes[0].legend(loc="lower right")
axes[1].legend(loc="lower right")
# fig.savefig("figures/three_odors/sum_cgammas_squared_gaussian_background.pdf", transparent=True)
plt.show()
plt.close()

In [ ]:
from simulfcts.plotting import plot_cbars_gamma_series

In [ ]:
# not interesting with gaussian degeneracy, not only two possible values of c_gamma. 
fig, ax, _ = plot_cbars_gamma_series(tser, cbars_gamma, skp=100, transient=50000)

In [ ]:
from simulfcts.plotting import plot_3d_series

In [ ]:
fig, ax = plot_3d_series(mbarser, dim_idx=[0, 1, 2], transient=10000, skp=1000)

# Annotate with vectors representing the odor components
orig = np.zeros([n_components, n_components])
xlim, ylim, zlim = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()
scale = 3
vecs = back_components.copy()
for i in range(n_components):
    vecs[i] = back_components[i] / np.sqrt(np.sum(back_components[i]**2)) * scale
ax.quiver(*orig, *(vecs[:, :3].T), color="k", lw=2.0)
ax.view_init(azim=45, elev=30)
ax.set(xlabel=r"$\overline{m}_1$", ylabel=r"$\overline{m}_2$", zlabel=r"$\overline{m}_3$")
# fig.savefig("figures/three_odors/points_fixes_ibcm_3_odeurs_gaussien.pdf", transparent=True)
plt.show()
plt.close()

In [ ]:
skp = 1
tini = 0
tmx = 100000# int(duration)#20000#int(duration)
tslice = slice(tini, tmx, skp)
fig, ax = plt.subplots()
w1_palette = sns.color_palette("Blues", n_colors=n_neurons)
w2_palette = sns.color_palette("Purples", n_colors=n_neurons)
w3_palette = sns.color_palette("Greens", n_colors=n_neurons)
#ax.plot(tser3[tslice], bkvecser3[tslice, 0], color="orange", alpha=0.5)
#ax.plot(tser3[tslice], bkvecser3[tslice, 1], color="blue", alpha=0.5)
#ax.plot(tser3[tslice], bkvecser3[tslice, 0], color="red", alpha=0.8)
#ax.plot(tser[tslice], thetaser[tslice, -1], color="orange", alpha=0.8)
for i in range(n_neurons-1):
    pass
    #ax.plot(tser3[tslice], mser3[tslice, i, 0], color=w1_palette[i], alpha=0.8)
    #ax.plot(tser3[tslice], mser3[tslice, i, 1], color=w2_palette[i], alpha=0.8)
    #ax.plot(tser3[tslice], mser3[tslice, i, 2], color=w3_palette[i], alpha=0.8)
ax.plot(tser[tslice], mser[tslice, -1, 0], color=w1_palette[-1], label="Neuron Component 0", alpha=0.8)
ax.plot(tser[tslice], mser[tslice, -1, 1], color=w2_palette[-1], label="Neuron Component 1", alpha=0.8)
ax.plot(tser[tslice], mser[tslice, -1, 2], color=w3_palette[-1], label="Neuron Component 2", alpha=0.8)


ax.set(xlabel="Time", ylabel="Inhibition neurons components")
ax.legend()
plt.show()
plt.close()

## Evolution of the inhibitory neurons' weights $\vec{w}_i$
Analytically, for a toy model with two neurons and a 2D mixture, I find that $\langle \vec{w}^j \rangle$ converges to $\frac{\alpha}{2\alpha + \beta}\vec{x}(\pm \sigma)$, i.e. to either input vector one standard deviation away from the mean input. 

### Analytical prediction of $\vec{w}$ for gaussian $\vec{x}$

TODO

In [ ]:
from simulfcts.plotting import plot_w_matrix

In [ ]:
# Plotting the time course of the dot products -- not interesting with gaussian degeneracy
# Unclear what it shows. 
fig, axes = plot_w_matrix(tser, wser, skp=500, lw=1.5)
        
plt.show()
plt.close()

## Background before and after inhibition

Not sure what is best to plot. I will try plotting the norm of the background as a function of time, without and with inhibition, on the same plot. 
However, this is not a good metric for, say, a background that alternates between equal norm odors, or a saturated background that just rotates. 
In that case, we need to compute the standard deviation of each ORN/PN and combine them, because the total activity norm would not fluctuate much. 

I also try, again, making small plots, one per component. 

TODO: These are still not publication-quality plots, but they will do for now. We need a rough draft of the manuscript before I really know what to plot. 

### Analytical prediction of $\vec{s}$ for gaussian $\vec{x}$ and many neurons
TODO

In [ ]:
from simulfcts.plotting import plot_background_norm_inhibition, plot_background_neurons_inhibition

In [ ]:
fig, ax, bknorm_ser, snorm_ser = plot_background_norm_inhibition(tser, bkvecser, sser)

# Compute noise reduction factor, annotate
transient = 50000
avg_bknorm = np.mean(bknorm_ser[transient:])
avg_snorm = np.mean(snorm_ser[transient:])
avg_reduction_factor = avg_snorm / avg_bknorm
std_bknorm = np.std(bknorm_ser[transient:])
std_snorm = np.std(snorm_ser[transient:])
std_reduction_factor = std_snorm / std_bknorm

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(avg_reduction_factor * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(std_reduction_factor * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(std_reduction_factor * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
#fig.savefig("figures/three_odors/inhibition_gaussian_background_norm_3odors.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, axes_mat, axes = plot_background_neurons_inhibition(tser, bkvecser, sser)
axes[-1].legend(loc="center right", bbox_to_anchor=(1.0, 0.6), fontsize=8, handlelength=1.5)
fig.tight_layout()
#fig.savefig("figures/three_odors/inhibition_gaussian_background_neurons_3odors.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# 3D plot of the original and inhibited odors, sampled sparsely in time
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Raw background
skp = 1000
tslice = slice(transient, None, skp)
ax.scatter(bkvecser[tslice, 0], bkvecser[tslice, 1], bkvecser[tslice, 2], color="r", label="Background")
# Compare to inhibition of the mean
mean_inhibition = bkvecser - np.mean(bkvecser[transient:], axis=0)*inhib_rates[0]/sum(inhib_rates)
ax.scatter(mean_inhibition[tslice, 0], mean_inhibition[tslice, 1], mean_inhibition[tslice, 2], 
           color="xkcd:light blue", label="Average subtraction")
ax.scatter(sser[tslice, 0], sser[tslice, 1], sser[tslice, 2], 
           color="b", label="IBCM inhibition")
ax.scatter(0, 0, 0, color="k", s=200, alpha=1)
ax.view_init(azim=150, elev=30)
ax.legend(loc="upper left", bbox_to_anchor=(1, 0.85))
plt.show()
plt.close()

## Response to a new odor
This part of the code only runs if the simulation above had ``n_dimensions > n_components``. 

The goal is to see whether a new odor, not linearly dependent of the ones in the background, also gets repressed close to zero, or produces an inhibited output noticeably different from the inhibited background, and still similar to the new odor vector, at least its component perpendicular to the background subspace.

I will test this property more carefully, over a statistical ensemble of new odors, once I have a more realistic model of odors themselves. Here, the goal is just to see if some component of the new odor is left after. To see this, I look at the mixture before and after inhibition, decomposed on the non-orthogonal basis of background + new odor. 

In [ ]:
def respond_new_odor_ibcm(odormix, typical_m, typical_w, coupling):
    # Compute activation of neurons with this new odor (new+background)
    # Given the IBCM and inhibitory neurons' current state 
    # (either latest or some average state of the neurons)
    c = typical_m.dot(odormix)
    cbar = c - coupling*(np.sum(c) - c)  # -c to cancel the subtraction of c[i] itself

    # New odor after inhibition by the network, ReLU activation on s
    # Inhibit with the mean cbar*wser, to see how on average the new odor will show
    new_output = relu_inplace(odormix - typical_w.dot(cbar))  # s = x - Wc
    return new_output

In [ ]:
new_odor = np.roll(back_components[0], shift=-1)  # Should be a new vector
full_basis = np.hstack([back_components.T, new_odor.reshape(-1, 1)])

# Mix new odor with background
current_background = back_components[1]
# new_odor = 0.5*new_odor + 0.5*np.sum(back_components, axis=0) / n_components  # Combine with mean background
new_odor = 0.5*new_odor + 0.5*current_background  # Combine with one component
# Inhibit with average m synapses and w
new_odor_after_inhibition_average = respond_new_odor_ibcm(new_odor, np.mean(mser[transient:], axis=0), 
                                                     np.mean(wser[transient:], axis=0), coupling_eta)
# Inhibit with latest m and w
new_odor_after_inhibition_latest = respond_new_odor_ibcm(new_odor, mser[-1], wser[-1], coupling_eta)

# Response to new odor after inhibition by removal of average background
# s = x - alpha/(alpha+beta)*mean_background
new_odor_after_average_subtract = relu_inplace(new_odor - (inhib_rates[0]/(sum(inhib_rates))
                                              * np.mean(bkvecser[transient:], axis=0)))

# Show components along the three background vectors plus the new odor vector (full_basis)
print("Decomposition on the basis of x_gamma")
print("Un-inhibited background", decompose_nonorthogonal_basis(current_background, full_basis))
print("Unhinibited new odor:", decompose_nonorthogonal_basis(new_odor, full_basis))
print("Average after IBCM inhibition:", decompose_nonorthogonal_basis(
                        new_odor_after_inhibition_average, full_basis))
print("Inhibition by average subtraction:", decompose_nonorthogonal_basis(
                        new_odor_after_average_subtract, full_basis))

# We indeed detect the new odor in the plane perpendicular to the inhibition. 
# Hence the more components of the new odor are not spanned by the old odors, the more
# we can detect it. 

# Inhibition of an alternating background
In this case we know, from original papers, that the fixed points of the IBCM model are orthogonal to all but one of the alternating components. It is thus easy to predict analytically the inhibition operated by this model. 

## Analytical predictions

### Fixed points of an IBCM neuron
In terms of the inhibited (bar) variables, equations are decoupled and the solutions reduce to those of a single IBCM neuron. The $K$ stable fixed points when there are $K$ possible values of $\vec{x}$ are defined by

$$ c_{\rho} = \vec{m} \cdot \vec{x}_{\rho} = \frac{1}{p_{\gamma}} \delta_{\rho \gamma} $$
    
where $\vec{x}_{\gamma}$ is the component for which the neuron is selective, $p_{\gamma}$ is the probability of that component, and $\delta_{\rho \gamma}$ is the Kronecker-delta symbol, equal to one only when $\vec{x} = \vec{x}_{\gamma}$. 

### Fixed point of inhibitory synaptic weights $\vec{w}$

TODO


### Instantaneous value of the inhibited background

TODO

In [ ]:
from modelfcts.backgrounds import update_alternating_inputs

In [ ]:
init_back_altern = [np.zeros(1), back_components[0]]  # Start with component 0
back_params_altern = [np.arange(n_components)/n_components, back_components]
init_synapses = 0.3*rgen_meta.random(size=[n_neurons, n_dimensions])

sim_res = integrate_inhib_ibcm_network(init_synapses, update_alternating_inputs, init_back_altern, ibcm_rates, 
            inhib_rates, back_params_altern, duration, deltat, seed=seed_from_gen(rgen_meta), noisetype="uniform")

t_ser_alt, bk_ser_alt, bkvec_ser_alt, m_ser_alt, cbar_ser_alt, _, w_ser_alt, s_ser_alt = sim_res

In [ ]:
# Calculate cgammas_bar and mbars
transient = 50000
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbar_ser_alt = m_ser_alt*(1.0 + coupling_eta) - coupling_eta*np.sum(m_ser_alt, axis=1, keepdims=True)
c_gammas_alt = m_ser_alt.dot(back_components.T)
cbars_gamma_alt = mbar_ser_alt.dot(back_components.T)

# Constaint 1: sum of c_gammas for each neuron should be 1/p_gamma, so here, =n_components
print("Comparison to analytical fixed points")
print("This matrix should have one non-zero element per row, equal to n_components={}:\n".format(n_components), 
      np.around(np.mean(cbars_gamma_alt[transient:], axis=0), 2))

### Time evolution of the solution

In [ ]:
fig, ax = plot_3d_series(mbar_ser_alt, dim_idx=[0, 1, 2], transient=10000, skp=1000)

# Annotate with vectors representing the odor components
orig = np.zeros([n_components, n_components])
xlim, ylim, zlim = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()
scale = 3
vecs = back_components.copy()
for i in range(n_components):
    vecs[i] = back_components[i] / np.sqrt(np.sum(back_components[i]**2)) * scale
ax.quiver(*orig, *(vecs[:, :3].T), color="k", lw=2.0)
ax.view_init(azim=30, elev=30)
ax.set(xlabel=r"$\overline{m}_1$", ylabel=r"$\overline{m}_2$", zlabel=r"$\overline{m}_3$")
# fig.savefig("figures/three_odors/points_fixes_ibcm_3_odeurs_alternant.pdf", transparent=True)
plt.show()
plt.close()

### Background components after inhibition
Total activity norm is constant, because alternating between background vectors of equal  norms. Need to compute the variance of each ORN/PN and show it is reduced in the inhibited layer. 

In [ ]:
fig, axes_mat, axes = plot_background_neurons_inhibition(t_ser_alt, bkvec_ser_alt, s_ser_alt, skp=300)

# Compute noise reduction factor, annotate
transient = 50000
avg_bknorm = np.mean(bkvec_ser_alt[transient:])
avg_snorm = np.mean(s_ser_alt[transient:])
avg_reduction_factor = avg_snorm / avg_bknorm
std_bknorm = np.std(bkvec_ser_alt[transient:])
std_snorm = np.std(s_ser_alt[transient:])
std_reduction_factor = std_snorm / std_bknorm

print("Mean activity of a projection neuron reduced to "
      + "{:.1f} % of input".format(avg_reduction_factor * 100))
print("Standard deviation of a projection neuron's activity reduced to "
      + "{:.1f} % of input".format(std_reduction_factor * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(std_reduction_factor * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

# Non-gaussian unimodal distribution
If there is a discrete number of fixed points when the $\nu_{\alpha}$ have a distribution with non-zero third moment, there should be a transition from a continuum of fixed points to this discrete case as we increase a parameter $\epsilon = \langle (\nu - \langle\nu\rangle)^3 \rangle$ above zero. 

To interpolate with a parameter $\epsilon$ from a gaussian distribution to a distribution with non-zero central third moment, one trick is to simulate $x$ as an Ornstein-Uhlenbeck process with zero mean, then take
$$ \nu = s + x + \epsilon x^2 $$
or, in the multivariate case, 
$$ \vec{\nu} = \vec{s} + \vec{x} + \epsilon \mathrm{diag}(\vec{x}) \vec{x} $$

If there are no correlations, we can treat each component $\nu_{\alpha}$ as a univariate case, and we then have a third moment of order $\epsilon$, with only lower-order corrections to the second moment and order $\epsilon$ corrections to the desired mean $s$:

$$ \langle \nu \rangle = s + \epsilon \sigma^2 $$
$$ \langle (\nu - \langle \nu \rangle)^2 \rangle = \sigma^2 + 2 \epsilon^2 \sigma^4 $$
$$ \langle (\nu - \langle \nu \rangle)^3 \rangle = 6 \epsilon \sigma^4 + 8 \epsilon^3 \sigma^6 $$

In [ ]:
from modelfcts.backgrounds import update_thirdmoment_kinputs

In [ ]:
# Reset some simulations parameters, others stay as before
# Initial synaptic weights: small positive noise near origin
rgen_meta = np.random.default_rng(seed=0xeefeb6e5f101c07cf9e80e95d5e8ecff)
init_synapses = 0.3*rgen_meta.random(size=[n_neurons, n_dimensions])

# Initial background vector and initial nu values
averages_nu = np.ones(n_components) / np.sqrt(n_components)
init_nu = np.zeros(n_components)
init_bkvec = averages_nu.dot(back_components)
# nus are first in the list of initial background params
init_back_list = [init_nu, init_bkvec]

## Compute the matrices in the Ornstein-Uhlenbeck update equation
# Update matrix for the mean term: 
# Exponential decay with time scale tau_nu over time deltat
tau_nu = 2.0  # Fluctuation time scale of the background nu_gammas (same for all)
update_mat_A = np.identity(n_components)*np.exp(-deltat/tau_nu)

# Steady-state covariance matrix
sigma2 = 0.09
correl_rho = 0.0
epsilon_nu = 0.2
steady_covmat = correl_rho * sigma2 * np.ones([n_components, n_components])  # Off-diagonals: rho
steady_covmat[np.eye(n_components, dtype=bool)] = sigma2  # diagonal: ones

# Cholesky decomposition of steady_covmat gives sqrt(tau/2) B
# Update matrix for the noise term: \sqrt(tau/2(1 - exp(-2*deltat/tau))) B
psi_mat = np.linalg.cholesky(steady_covmat)
update_mat_B = np.sqrt(1.0 - np.exp(-2.0*deltat/tau_nu)) * psi_mat

back_params_3 = [update_mat_A, update_mat_B, back_components, averages_nu, epsilon_nu]

In [ ]:
# m_init, update_bk, bk_init, ibcm_params, inhib_params, bk_params, tmax, dt, seed=None, noisetype="normal"
sim_results = integrate_inhib_ibcm_network(init_synapses, update_thirdmoment_kinputs, init_back_list, ibcm_rates, 
                inhib_rates, back_params_3, duration, deltat, seed=seed_from_gen(rgen_meta), noisetype="normal")
tser3, nuser3, bkvecser3, mser3, cbarser3, _, wser3, sser3 = sim_results

In [ ]:
# Calculate cgammas_bar and mbars
transient = 50000
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser3 = mser3*(1.0 + coupling_eta) - coupling_eta*np.sum(mser3, axis=1, keepdims=True)
c_gammas3 = mser3.dot(back_components.T)
cbars_gamma3 = mbarser3.dot(back_components.T)

# For quantitative comparison to analytical values, see below:
# requires computing corrections due to third moment. 
sums_cbars_gamma_3 = np.sum(cbars_gamma3, axis=2)
sums_cbars_gamma2_3 = np.sum(cbars_gamma3*cbars_gamma3, axis=2)

## Time evolution of IBCM neurons

In [ ]:
fig, ax = plot_3d_series(mbarser3, dim_idx=[0, 1, 2], transient=10000, skp=1000)

# Annotate with vectors representing the odor components
orig = np.zeros([n_components, n_components])
xlim, ylim, zlim = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()
scale = 3
vecs = back_components.copy()
for i in range(n_components):
    vecs[i] = back_components[i] / np.sqrt(np.sum(back_components[i]**2)) * scale
ax.quiver(*orig, *(vecs[:, :3].T), color="k", lw=2.0)
ax.view_init(azim=30, elev=30)
ax.set(xlabel=r"$\overline{m}_1$", ylabel=r"$\overline{m}_2$", zlabel=r"$\overline{m}_3$")
# fig.savefig("figures/three_odors/points_fixes_ibcm_3_odeurs_non-gaussien.pdf", transparent=True)
plt.show()
plt.close()

## Background components after inhibition

In [ ]:
fig, ax, bknorm_ser3, snorm_ser3 = plot_background_norm_inhibition(tser3, bkvecser3, sser3)

# Compute noise reduction factor, annotate
transient = 50000
avg_bknorm = np.mean(bknorm_ser3[transient:])
avg_snorm = np.mean(snorm_ser3[transient:])
avg_reduction_factor = avg_snorm / avg_bknorm
std_bknorm = np.std(bknorm_ser3[transient:])
std_snorm = np.std(snorm_ser3[transient:])
std_reduction_factor = std_snorm / std_bknorm

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(avg_reduction_factor * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(std_reduction_factor * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(std_reduction_factor * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
#fig.savefig("figures/three_odors/inhibition_non-gaussian_background_norm_3odors.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, axes_mat, axes = plot_background_neurons_inhibition(tser3, bkvecser3, sser3, skp=100)

# Compute noise reduction factor, annotate
transient = 50000
avg_bknorm = np.mean(bkvecser3[transient:])
avg_snorm = np.mean(sser3[transient:])
avg_reduction_factor = avg_snorm / avg_bknorm
std_bknorm = np.std(bkvecser3[transient:])
std_snorm = np.std(sser3[transient:])
std_reduction_factor = std_snorm / std_bknorm

print("Mean activity of a projection neuron reduced to "
      + "{:.1f} % of input".format(avg_reduction_factor * 100))
print("Standard deviation of a projection neuron's activity reduced to "
      + "{:.1f} % of input".format(std_reduction_factor * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(std_reduction_factor * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
#fig.savefig("figures/three_odors/inhibition_non-gaussian_background_neurons_3odors.pdf", 
#           bbox_inches="tight")
plt.show()
plt.close()

## Analytical fixed point prediction compared to simulations
Here, it is harder to make analytical progress. I could not solve the IBCM steady-state equation exactly yet, but I could show, using symmetries of the system of equations, that the set of variables $c_{\alpha} = \vec{m} \cdot \vec{x}_{\alpha}$, at a given fixed point, can take either all the same value (leading to an unstable fixed point), or each take one of two values (which I am not able to compute explicitly). It is not possible to have three or more $c_{\alpha}$ differing from each other (for instance, it is not possible to have $c_{\alpha} \neq c_{\beta} \neq c_{\gamma}$), because the sum of any two $c_{\alpha}, c_{\beta}$ that are different from one another must be equal to the same one constant. 

This is confirmed numerically below (with some error): one component $c_{\alpha}$ is large and positive, while other $c_{\beta}$, $\forall \beta \neq \alpha$, are small and negative and equal to each other, with of course some variability due to correlations neglected in the analytical calculations, and to long excursions in the simulations due to having a flat direction in the landscape (the slow manifold). 

In [ ]:
from modelfcts.ibcm_analytics import fixedpoint_thirdmoment_perturbtheory, fixedpoint_thirdmoment_exact
def fixedpoint_thirdmoment_xy(*args, **kwargs):
    # The function returns cd and u2 too, here we just want x, y. 
    return fixedpoint_thirdmoment_perturbtheory(*args, **kwargs)[:2]


In [ ]:
### Analytical values
# Compute actual mean, variance and third moment for nu = s + nu_gauss + epsilon*nu_gauss^2
variance_nu3 = sigma2 + 2*(epsilon_nu*sigma2)**2
mean_nu3 = averages_nu[0] + epsilon_nu*sigma2
thirdmoment = 6*epsilon_nu*sigma2**2 + 8*(epsilon_nu*sigma2)**3

print("Expected moments of the background mixture coefficients:")
print("<(nu - <nu>)^3> =", thirdmoment)
print("<(nu - <nu>)^2> =", variance_nu3)
print("<nu> =", mean_nu3)

# Check that empirical statistics of nu match analytical expectations
print("Compare to empirical moments:")
fullnuser3 = averages_nu.reshape(1, -1) + nuser3 + epsilon_nu*nuser3**2
empirical_thirdmoment = np.mean((fullnuser3[:, 0] - np.mean(fullnuser3[:, 0]))**3)
print("<(nu - <nu>)^3> =", empirical_thirdmoment)

empirical_secondmoment = np.mean((fullnuser3[:, 0] - np.mean(fullnuser3[:, 0]))**2)
print("<(nu - <nu>)^2> =", empirical_secondmoment)

print("<nu> =", np.mean(fullnuser3[:, 0]))  # First moment


# Compute the analytical predictions of the fixed points
res = fixedpoint_thirdmoment_xy(mean_nu3, variance_nu3, thirdmoment, 1, 2, m3=1.0, order=1)
cgamma1_analytical_order1, cgamma2_analytical_order1 = res
print("c_\gamma values, order 1:", cgamma1_analytical_order1, cgamma2_analytical_order1)

# Order zero
res = fixedpoint_thirdmoment_xy(mean_nu3, variance_nu3, thirdmoment, 1, 2, m3=1.0, order=0)
cgamma1_analytical_order0, cgamma2_analytical_order0 = res
print("c_\gamma values, zeroth order:", cgamma1_analytical_order0, cgamma2_analytical_order0)

# Exact solution
res = fixedpoint_thirdmoment_exact([mean_nu3, variance_nu3, thirdmoment], 1, 2)
cgamma1_exact, cgamma2_exact, _, _ = res
print("c_\gamma values, exact:", cgamma1_exact, cgamma2_exact)

### Empirical values
# Average value of the m.x_a for each neuron, for each x_a
# avg_m indexed [neuron, dimension], back_components indexed [component, dimension]
# So need to take m.dot(x_a.T)
transient = 60000
avg_mbar = np.mean(mbarser3[transient:], axis=0)
steadystate_c_gamma = avg_mbar.dot(back_components.T)
print(steadystate_c_gamma)

In [ ]:
# Make a nice graph of the time series of the c_gamma in each neuron vs the two possible 
# values for c_gamma as prediction from perturbation theory
# These time series were computed above in cbar_gammas3
# Also plot the average c_gammas of each neuron in addition to their time series? 
# These were computed above too in steadystate_c_gamma
fig, ax, n_closest = plot_cbars_gamma_series(tser3, cbars_gamma3, skp=100, transient=60000)

# Plot analytical prediction of the two possible values for any c_gamma:
ax.axhline(cgamma1_analytical_order1, color="y", ls="-", 
           label=r"Unique $\bar{c}_\gamma$, 1st-order", lw=3.)
ax.axhline(cgamma2_analytical_order1, color="y", ls="--", 
           label=r"Other $\bar{c}_\gamma$s, 1st-order", lw=3.)

# Compare to zeroth order prediction
#ax.axhline(cgamma1_analytical_order0, color="grey", ls="-", 
#           label=r"Unique $\bar{c}_\gamma$, 0th-order", lw=3.)
#ax.axhline(cgamma2_analytical_order0, color="grey", ls="--", 
#           label=r"Other $\bar{c}_\gamma$s, 0th-order", lw=3.)
# Compare to exact prediction
ax.axhline(cgamma1_exact, color="grey", ls="-", 
           label=r"Unique $\bar{c}_\gamma$, exact", lw=3.)
ax.axhline(cgamma2_exact, color="grey", ls="--", 
           label=r"Other $\bar{c}_\gamma$s, exact", lw=3.)

ax.legend(fontsize=9, loc="upper left", bbox_to_anchor=(1, 1))
fig.set_size_inches(4.0, 4.)
#fig.savefig("figures/three_odors/cgammas_thirdmoment_background_perturbation_theory_prediction.pdf", 
#   transparent=True, bbox_inches="tight")
plt.show()
plt.close()

## Analytical stability
I can compute analytically the jacobian of the average dynamics (for i.i.d. background concentrations, weakly non-gaussian), but I am not able to compute its eigenvalues or to apply Routh-Hurwitz criterion to determine stability. So I will compute the jacobian based on analytical expressions, then diagonalize it numerically. 

There should be $n_B + 1$ non-zero eigenvalues, the rest ($n_O - n_B - 1$) should be zero because the jacobian is entirely made up of outer products of background odor vectors. 

I will compute the jacobian for various $k_1$ and $k_2$ (numbers of components with specific/non-specific dot product values). The goal is to show the only stable fixed points have $k_1 = 1$ and $k_2 = n_B - 1$. 

In [ ]:
def jacobian_fixedpoint_thirdmoment(moments, ibcm_params, which_specif, back_comps, m3=1.0, order=1):
    """ which_specif: boolean array equal to True for specific gammas. """
    ## 1. Evaluate x, y, cd, u^2 at the fixed point
    # From the list of c_gammas which are specific, count k1 and k2
    assert which_specif.size == back_comps.shape[0]
    which_specif = which_specif.astype(bool)
    k1 = np.sum(which_specif.astype(bool))
    k2 = which_specif.size - k1

    avgnu, variance, epsilon = moments
    mu, tau_theta, eta = ibcm_params
    c_sp, c_nsp, cd, u2 = fixedpoint_thirdmoment_perturbtheory(avgnu,
                        variance, epsilon, k1, k2, m3=m3, order=order)
    x_d = avgnu * np.sum(back_comps, axis=0)
    cgammas_vec = np.where(which_specif, c_sp, c_nsp)
    # 2. Evaluate the jacobian blocks
    n_dims = x_d.shape[0]
    n_comp = k1 + k2
    jac = np.zeros([n_dims + 1, n_dims + 1])
    # Scalar element
    jac[-1, -1] = -1.0 / tau_theta
    # Vector blocks
    avg_cx = cd * x_d + variance * cgammas_vec.dot(back_comps)
    # Last column
    jac[:n_dims, -1] = 2.0 / tau_theta * avg_cx
    # Last row
    jac[-1, :n_dims] = -mu * avg_cx
    # Matrix block
    theta_ss = cd**2 + variance * u2
    x_gammas_outer = back_comps[:, :, None] * back_comps[:, None, :]
    xd_outer = np.outer(x_d, x_d)
    xd_xgammas_outer = x_d[None, :, None] * back_comps[:, None, :]
    avg_xx = xd_outer + variance*np.sum(x_gammas_outer, axis=0)
    avg_cxx = (cd * avg_xx
                + variance*np.sum(cgammas_vec[:, None, None]*xd_xgammas_outer, axis=0)
                + variance*np.sum(cgammas_vec[:, None, None]*xd_xgammas_outer, axis=0).T
                + epsilon*m3*np.sum(cgammas_vec[:, None, None]*x_gammas_outer, axis=0)
            )
    jac[:-1, :-1] = mu * (2*avg_cxx - theta_ss*avg_xx)

    # Build the complete matrix
    return jac


In [ ]:
# Compute jacobian for various $k_1$ and $k_2$. 
# Without loss of generality, assume it's the first 
# $k_1$ x_gammas which elicit a specific response.  
moments_3 = [mean_nu3, variance_nu3, thirdmoment]
for k1 in range(0, n_components):
    # moments, ibcm_params, which_specif, back_comps, m3=1.0, order=1
    which_specific = np.zeros(n_components, dtype=bool)
    which_specific[:k1] = True
    jacob = jacobian_fixedpoint_thirdmoment(moments_3, ibcm_rates, 
                    which_specific, back_components, m3=1.0, order=1)
    print("For k1 = {}, jacobian matrix x 1000:\n".format(k1), np.round(jacob*1e3, 2))
    eigenvalues = np.linalg.eigvals(jacob)
    sort_order = np.argsort(np.abs(np.real(eigenvalues)))[::-1]
    eigenvalues = eigenvalues[sort_order]
    print("For k1 = {}, eigenvalues x 1000:\n".format(k1), np.round(eigenvalues*1e3, 6))
    if np.all(np.real(eigenvalues[:n_components+1]) < 0):
        print("This is a stable fixed point!")
    else:
        print("This fixed point is unstable")
    print()

### Fixed points of the inhibition vectors w
TODO

In [ ]:
# TODO
def fixedpoint_wvec_given_cgammas(cgammas, back_comps, back_stats, avg_rates):
    """ For the special case where all $\nu$ are iid with mean and variance
    given in back_stats. 
    
    Args:
        cgammas (np.ndarray): dot products at the fixed point 
            of mbar with each background component. 
            Shape [..., component]
        back_comps (np.ndarray): background vectors x_gammas, 
            indexed [component, dimension]
        back_stats (list of 2 floats): average nu, sigma^2. 
        avg_rates (list of 2 floats): alpha, beta
    
    Returns:
        wvec (np.ndarray): 1d steady-state vector, 
            shape [..., component]
    """
    raise NotImplementedError()
    avgnu, varinu = back_stats
    alph, bet = avg_rates
    cbard = avgnu * np.sum(cgammas, axis=-1, keepdims=True)
    term_mean = avgnu * cbard * np.sum(back_comps, axis=0)
    term_vari = varinu * np.dot(cgammas, back_comps)  # Sum over gamma (indexing background components)
    return alph / (alph*cbard + bet) * (term_mean + term_vari)
    

In [ ]:
all_fixed_cgammas = np.ones([n_components, n_components]) * cgamma2_analytical_order1
all_fixed_cgammas[np.diag_indices(n_components)] = cgamma1_analytical_order1
wvecs_analytical = fixedpoint_wvec_given_cgammas(all_fixed_cgammas, back_components, 
                                                 (mean_nu3, variance_nu3), inhib_rates)
print(wvecs_analytical)

all_fixed_cgammas_zeroth = np.ones([n_components, n_components]) * cgamma2_analytical_order0
all_fixed_cgammas_zeroth[np.diag_indices(n_components)] = cgamma1_analytical_order0
wvecs_analytical_zeroth = fixedpoint_wvec_given_cgammas(all_fixed_cgammas_zeroth, back_components, 
                                                 (mean_nu3, variance_nu3), inhib_rates)

In [ ]:
# Make a nice graph of the time series of the wvec in each neuron vs the possible 
# values for wvec components (exploit symmetry: fixed points are permutations of same 4 values)
# These time series were computed above in wser3
fig, ax = plt.subplots()

# Colors: one palette per component
palettes = ["Blues", "Purples", "Greens", "Oranges", "Reds"]
if len(palettes) < n_dimensions:
    raise ValueError("Need to define at least {} more palettes!".format(n_dimensions-len(palettes)))
all_colors = [sns.color_palette(p, n_colors=n_neurons) for p in palettes]

tsli = slice(0, None, 200)
for n in range(n_neurons):
    for i in range(n_dimensions):
        lbl = "Dimension {}".format(i) if n == n_neurons-1 else None
        ax.plot(tser3[tsli], wser3[tsli, n, i], lw=1.25, color=all_colors[i][n], alpha=0.75, label=lbl)

# Plot analytical prediction of the two possible values for any c_gamma:
for i in range(n_dimensions):
    lbl = r"Analytical $\vec{w}$, order 0" if i == n_dimensions - 1 else ""
    ax.axhline(wvecs_analytical_zeroth[0, i], color="grey", ls="--", label=lbl, lw=2.5)
    lbl = r"Analytical $\vec{w}$, order 1" if i == n_dimensions - 1 else ""
    ax.axhline(wvecs_analytical[0, i], color="y", ls="--", label=lbl, lw=2.5)

ax.set(xlabel="Time (steps)", ylabel=r"$\vec{w}$ components")
ax.legend(fontsize=9)
# fig.savefig("figures/three_odors/wvecs_thirdmoment_background_perturbation_theory_prediction.pdf", transparent=True)
plt.show()
plt.close()